In [ ]:
# !pip install peft trl bitsandbytes transformers accelerate

In [ ]:
#restart runtime.

https://github.com/meetrais/LLM-Fine-Tuning/blob/main/finetune_mistral_7b.py, https://www.reddit.com/r/LocalLLaMA/comments/17wou8y/comment/k9j7718/

In [ ]:
import transformers  as transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig

from trl import SFTTrainer
from transformers import BitsAndBytesConfig, TrainingArguments
import torch
import torch.nn as nn
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

model_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
text = "Capital of USA is"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)

bnb_config = BitsAndBytesConfig(
    # quant_method= QuantizationMethod.BITS_AND_BYTES,
    load_in_8bit= False,
    llm_int8_threshold= 6.0,
    llm_int8_skip_modules= None,
    llm_int8_enable_fp32_cpu_offload= False,
    llm_int8_has_fp16_weight= False,
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

#Freezing the original weights
for param in model.parameters():
    param.requires_grad = False
    if param.ndim ==1:
        param.data = param.data.to(torch.float32)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

#Setting up the LoRa Adapters
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

config  = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias = 'none',
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6815744 || all params: 3758886912 || trainable%: 0.18132346515244138


In [ ]:
dataset_name = "gathnex/Gath_baize"
dataset = load_dataset(dataset_name, split="train[:1000]")
dataset["chat_sample"][0]

# def merge_colunms(example):
#     example['prediction'] = example['quote'] + " ->: " + str(example["tags"])
#     return example

# print(dataset["chat_sample"][0])

# data['train'] = data['train'].map(merge_colunms)
# print(data['train'][0])

# #data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

# #print(data)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['chat_sample', 'dataset_origin'],
    num_rows: 1000
})

In [ ]:
#Training
training_arguments = TrainingArguments(
    output_dir= "./results",
    num_train_epochs= 10,
    per_device_train_batch_size= 8,
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 100,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= True,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "constant"
)
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length= 20,
    dataset_text_field="chat_sample",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

model.config.use_cache = False
trainer.train()

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss
30,1.416300
60,0.318900
90,0.230500
120,0.198700
150,0.192100
180,0.191800
210,0.190400
240,0.179800
270,0.191100


Step,Training Loss
30,1.416300
60,0.318900
90,0.230500
120,0.198700
150,0.192100
180,0.191800
210,0.190400
240,0.179800
270,0.191100
300,0.187600


In [ ]:



#print(model)

config  = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias = 'none',
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj","lm_head"]
)

# model = get_peft_model(model, config)
outputs = model.generate(**inputs, max_new_tokens=30)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

"""
model.push_to_hub("meetrais/finetuned_mistral_7b",
                  token="Your-Hugging-Face-Token-Here",
                  commit_message="basic training",
                  private=True)

"""